In [9]:
#Librerías a usar
%matplotlib inline
import numpy as np
import pandas as pd
import random
from collections import Counter
import random
import uuid

In [10]:
#Tabla de datos filtrada 
#Contiene distancia entre cada departamento (en km y en número de horas), incidencia de cáncer por departamento 
arch2 = pd.read_csv('./BD_Simulacion_SinBAVc.csv')
arch2['people_count_round']=round(arch2['people_count'])
arch2.head()
#arch2.info()

,Departamento,population_mm,population_ratio,latitud,longitud,Hours ARAUCA,Distance ARAUCA,Hours ATLANTICO,Distance ATLANTICO,Hours BOLIVAR,...,TAE melanoma. hombres,TAE cáncer colorrectal. mujeres,TAE linfoma no Hodgkin. mujeres,TAE linfoma Hodgkin. hombres,TAE linfoma Hodgkin. mujeres,TAE melanoma. mujeres,net,people_count,net_norm,people_count_round
0,CUNDINAMARCA,2.92,2920000,4.57937,-74.21682,13.923056,708.167,17.075000,996.623,18.508889,...,3.5,9.9,5.7,0.9,0.2,3.4,170.9,4990.28,499.028,4990.0
1,ATLANTICO,2.53,2530000,10.96854,-74.78132,18.233611,948.746,0.000000,0.000,2.435833,...,0.9,11.9,4.8,1.1,0.5,0.5,190.7,4824.71,482.471,4825.0
2,SANTANDER,2.18,2180000,7.12539,-73.11980,13.130000,444.598,10.255000,579.202,11.688889,...,3.0,12.5,4.7,0.5,0.1,3.3,184.9,4030.82,403.082,4031.0
3,BOLIVAR,2.07,2070000,10.39972,-75.51444,19.801111,1001.901,2.445556,131.001,0.000000,...,0.3,8.1,3.5,0.8,0.6,0.8,150.3,3111.21,311.121,3111.0
4,CORDOBA,1.78,1780000,8.74798,-75.88143,19.439444,986.047,6.008056,348.952,4.488056,...,1.1,5.6,3.6,1.1,0.7,0.6,134.9,2401.22,240.122,2401.0


In [11]:
#Definición de objetos 

#Definición de la clase 'City'. Tiene como atributos el nombre, si tiene máquina y su identificación y la población. Éste último
#valor está dado por la incidencia de cáncer por departamento. 
class City:
    def __init__(self, name,population,machine=None):
        self.name = name
        self.machine = machine
        self.population = population
    
    def __repr__(self):
        return f"{self.name}, machine: {self.machine}"
        

LIMIT = 100
FREE = 1
OCCUPIED = 2
#Definición de la clase 'Machine'. Puede estar ocupada o libre. Tiene el registro de los pacientes atentidos. 
class Machine:
    def __init__(self):
        self.state = FREE
        self.treated = []
        
    def __repr__(self):
        return "MACHINE"
    
SEEKING_MACHINE = 1
TREATED = 2
#Definición de la clase 'person'. Tiene como atributos el origen, los lugares visitados, el estado, el destino en ese momento
#y el tiempo restante del viaje. 
class Person:
    def __init__(self, origin, destination,timeD):
        self.id = uuid.uuid1()
        self.origin = origin
        self.visited = [origin]
        self.status = SEEKING_MACHINE
        self.destination = destination
        self.tripRemaining = timeD # replace with value from table
#treated_people=[]   
#pers=[]
class World():
    #función para incializar las clases
    def __init__(self,delta,people):
        self.delta=delta
        self.people = people
        #self.cities = cities
        #self.step = step
    
    #función que la actualización de estados en cada paso de la simulación 
    #Se actualiza: el tiempo restante para llegar a la ciudad de destino, se cambia el estado de ciudades visitadas de ser
    #necesario, se actualizan el número de pacientes atentidos por máquina, se asigna un nuevo origen a la persona de ser 
    #necesario. 
    def step(self):
        #array para guardar el número de pacientes que lograron ser atendidos con la iteración
        
        for person in self.people:
            
            if person.tripRemaining >= 0: 
                person.tripRemaining = person.tripRemaining - self.delta
            else:
                cityD = person.destination
                person.visited.append(cityD)
                
                if cityD.machine is not None: #si llega al destino se aplica este condicional
                    if len(cityD.machine.treated) < LIMIT:
                        print(f"Person {person.id} comming from {person.origin.name}")
                        person.status = TREATED #cambia el estado si hay cupos en la máquina
                        person
                        if (person in treated_people):
                            pass
                        else:
                            cityD.machine.treated.append(person)
                            treated_people.append(person)   #hacer un array de todas las personas tratadas a nivel nacional   
                    if len(cityD.machine.treated) == LIMIT: #condicional para decidir qué pasa cuando se llena la máquina
                        new_ori=cityD.name
                        person.origin = new_ori
                        tablaf_newD = tablasCorr(new_ori)
                        time_d=min(tablaf_newD)
                        n_ciudad_destino=tablaf_newD[tablaf_newD==time_d].index.values[0].split('s ')[1]    
                
                else:
                    #ir para otro lado si la ciudad no tiene máquinas 
                    #cityD.machine.status = OCCUPIED
                    new_ori=cityD.name
                    person.origin = new_ori
                    tablaf_newD = tablasCorr(new_ori)
                    time_d=min(tablaf_newD)
                    n_ciudad_destino=tablaf_newD[tablaf_newD==time_d].index.values[0].split('s ')[1]
                
            
        
        return treated_people
    

In [12]:
#inicialización
# Función que asigna un número de máquinas que entra por parámetro 
def Machine_Distribution(MACHINE_COUNT):
    for cityWithMachine in random.sample(cities, MACHINE_COUNT):
        print(cityWithMachine)
        cityWithMachine.machine = Machine()
    return cities

#asignación de máquinas (aleatoria). Función que asigna n máquinas aleatorias a la lista lista de departamentos de arch2.csv
cities = []
for depa in arch2['Departamento']:
    #city = depa
    population = arch2.set_index('Departamento').loc[depa].people_count_round
    city = City(depa,population)
    cities.append(city)
    
MACHINE_COUNT=20
maquinas=Machine_Distribution(MACHINE_COUNT)
print(cities)


#Filtrar tabla para tener únicamente como destinos los departamentos que tiene máquinas. 
columnas_ver=[x.name for x in cities if x.machine !=None]
columnas_ver=['Hours '+x for x in columnas_ver]
tablaf=arch2.set_index('Departamento')[columnas_ver]

# funcion para eliminar de tablaf los ceros si los hay
def tablasCorr(ciudad):
    newTabla2=[]
    t_d=min(tablaf.loc[ciudad])
    if t_d<0.1:
        newTabla=tablaf.loc[ciudad]
        newTabla2=newTabla.drop(labels=['Hours '+ciudad])
    else:
        newTabla2=tablaf.loc[ciudad]
    return newTabla2


#definición de personas
personas = []
personas_def=[]


#inicializar personas
for city in tablaf.index:
    tablaf_loc = tablasCorr(city)
    time_d=min(tablaf_loc)
    n_ciudad_destino=tablaf_loc[tablaf_loc==time_d].index.values[0].split('s ')[1]    
    
    obj_ciudad_o=[x for x in cities if x.name== city][0]
    obj_ciudad_d=[x for x in cities if x.name== n_ciudad_destino][0]
    
    for person in range(int(obj_ciudad_o.population)):
        personas.append(Person(obj_ciudad_o,obj_ciudad_d,time_d))
        
personas_def= personas.copy()
np.random.shuffle(personas_def) #cambiar orden aleatoriamente de las personas

#inicializar mundo


    

CUNDINAMARCA, machine: None
SANTANDER, machine: None
NORTE DE SANTANDER, machine: None
SUCRE, machine: None
CHOCO, machine: None
PUTUMAYO, machine: None
CALDAS, machine: None
LA GUAJIRA, machine: None
CAQUETA, machine: None
HUILA, machine: None
BOYACA, machine: None
BOLIVAR, machine: None
NARIÑO, machine: None
CASANARE, machine: None
ATLANTICO, machine: None
VICHADA, machine: None
META, machine: None
RISARALDA, machine: None
ARAUCA, machine: None
CORDOBA, machine: None
[CUNDINAMARCA, machine: MACHINE, ATLANTICO, machine: MACHINE, SANTANDER, machine: MACHINE, BOLIVAR, machine: MACHINE, CORDOBA, machine: MACHINE, NARIÑO, machine: MACHINE, NORTE DE SANTANDER, machine: MACHINE, MAGDALENA, machine: None, TOLIMA, machine: None, BOYACA, machine: MACHINE, CESAR, machine: None, HUILA, machine: MACHINE, META, machine: MACHINE, CALDAS, machine: MACHINE, RISARALDA, machine: MACHINE, SUCRE, machine: MACHINE, LA GUAJIRA, machine: MACHINE, QUINDIO, machine: None, CHOCO, machine: MACHINE, CASANARE, ma

In [ ]:
n_pasos = 300
delta = 0.1 #tiempo de cada paso en horas
#ciudad de origen de las personas 
#for persona in personas_def:
#    print(persona.origin.name)

treated_people=[] 
#ejecución de la simulación 
print('resultado de la simulación')
mundo=World(delta,personas_def)
for i in range(n_pasos):
    print("estoy en el paso " + str(i))
    bla=mundo.step()
    print(len(treated_people))



print('*********')


resultado de la simulación
estoy en el paso 0
0
estoy en el paso 1
0
estoy en el paso 2
0
estoy en el paso 3
0
estoy en el paso 4
0
estoy en el paso 5
0
estoy en el paso 6
0
estoy en el paso 7
0
estoy en el paso 8
0
estoy en el paso 9
0
estoy en el paso 10
0
estoy en el paso 11
0
estoy en el paso 12
Person 73112c69-a663-11ec-a958-f8e4e3a62943 comming from QUINDIO
Person 731179dc-a663-11ec-984a-f8e4e3a62943 comming from QUINDIO
Person 73112c0c-a663-11ec-9d75-f8e4e3a62943 comming from QUINDIO
Person 7310dded-a663-11ec-9bd6-f8e4e3a62943 comming from QUINDIO
Person 7311107d-a663-11ec-9d40-f8e4e3a62943 comming from QUINDIO
Person 73117a53-a663-11ec-9e14-f8e4e3a62943 comming from QUINDIO
Person 73112c1b-a663-11ec-8379-f8e4e3a62943 comming from QUINDIO
Person 73112bbc-a663-11ec-b6ed-f8e4e3a62943 comming from QUINDIO
Person 7311a111-a663-11ec-a1fb-f8e4e3a62943 comming from QUINDIO
Person 7310de0a-a663-11ec-8fe7-f8e4e3a62943 comming from QUINDIO
Person 73115309-a663-11ec-aee8-f8e4e3a62943 commi

200
estoy en el paso 14
200
estoy en el paso 15
200
estoy en el paso 16
200
estoy en el paso 17
200
estoy en el paso 18
200
estoy en el paso 19
200
estoy en el paso 20
Person 72e8eed4-a663-11ec-b0a0-f8e4e3a62943 comming from MAGDALENA
Person 72e82d89-a663-11ec-91de-f8e4e3a62943 comming from MAGDALENA
Person 72e98e3b-a663-11ec-aed6-f8e4e3a62943 comming from MAGDALENA
Person 72e9b170-a663-11ec-ad83-f8e4e3a62943 comming from MAGDALENA
Person 72e82d2d-a663-11ec-a95e-f8e4e3a62943 comming from MAGDALENA
Person 72e8eedf-a663-11ec-8ad7-f8e4e3a62943 comming from MAGDALENA
Person 72e91584-a663-11ec-b17c-f8e4e3a62943 comming from MAGDALENA
Person 72e93ca4-a663-11ec-91a7-f8e4e3a62943 comming from MAGDALENA
Person 72e82cfd-a663-11ec-b8da-f8e4e3a62943 comming from MAGDALENA
Person 73008a8c-a663-11ec-bd4f-f8e4e3a62943 comming from MAGDALENA
Person 72e9155d-a663-11ec-86b6-f8e4e3a62943 comming from MAGDALENA
Person 7300b195-a663-11ec-99be-f8e4e3a62943 comming from MAGDALENA
Person 72e93c44-a663-11ec-ba

Person 730f5727-a663-11ec-bb39-f8e4e3a62943 comming from SUCRE
Person 72e23cee-a663-11ec-8e56-f8e4e3a62943 comming from CORDOBA
Person 730fcd97-a663-11ec-99fa-f8e4e3a62943 comming from SUCRE
Person 72e2d85e-a663-11ec-a94d-f8e4e3a62943 comming from CORDOBA
Person 730fa643-a663-11ec-ba28-f8e4e3a62943 comming from SUCRE
Person 72e1c730-a663-11ec-a362-f8e4e3a62943 comming from CORDOBA
Person 730f823f-a663-11ec-8f3b-f8e4e3a62943 comming from SUCRE
Person 72e23ca2-a663-11ec-8f30-f8e4e3a62943 comming from CORDOBA
Person 730fcd1a-a663-11ec-9d66-f8e4e3a62943 comming from SUCRE
Person 730fcd92-a663-11ec-92ec-f8e4e3a62943 comming from SUCRE
Person 730f8266-a663-11ec-8565-f8e4e3a62943 comming from SUCRE
Person 730f8225-a663-11ec-b57d-f8e4e3a62943 comming from SUCRE
Person 730f56b1-a663-11ec-9f35-f8e4e3a62943 comming from SUCRE
Person 730f8215-a663-11ec-bbde-f8e4e3a62943 comming from SUCRE
Person 730fa62e-a663-11ec-ae75-f8e4e3a62943 comming from SUCRE
Person 730f2e2b-a663-11ec-b7d3-f8e4e3a62943 com

600
estoy en el paso 26
600
estoy en el paso 27
600
estoy en el paso 28
600
estoy en el paso 29
600
estoy en el paso 30
Person 72d2b887-a663-11ec-83e5-f8e4e3a62943 comming from CUNDINAMARCA
Person 72bdb67b-a663-11ec-bbc1-f8e4e3a62943 comming from CUNDINAMARCA
Person 72bd6834-a663-11ec-b3e3-f8e4e3a62943 comming from CUNDINAMARCA
Person 72be5279-a663-11ec-befd-f8e4e3a62943 comming from CUNDINAMARCA
Person 72d46429-a663-11ec-8161-f8e4e3a62943 comming from CUNDINAMARCA
Person 72d43d4e-a663-11ec-a35c-f8e4e3a62943 comming from CUNDINAMARCA
Person 72d32d43-a663-11ec-80e1-f8e4e3a62943 comming from CUNDINAMARCA
Person 72be0498-a663-11ec-916d-f8e4e3a62943 comming from CUNDINAMARCA
Person 72d30692-a663-11ec-91b5-f8e4e3a62943 comming from CUNDINAMARCA
Person 72bccc7b-a663-11ec-9931-f8e4e3a62943 comming from CUNDINAMARCA
Person 72bcccd7-a663-11ec-85ac-f8e4e3a62943 comming from CUNDINAMARCA
Person 72bc57cf-a663-11ec-84f8-f8e4e3a62943 comming from CUNDINAMARCA
Person 72beef28-a663-11ec-816e-f8e4e3a62

800
estoy en el paso 33
800
estoy en el paso 34
800
estoy en el paso 35
Person 73081cc3-a663-11ec-8ad2-f8e4e3a62943 comming from CESAR
Person 73084368-a663-11ec-94c0-f8e4e3a62943 comming from CESAR
Person 7306e2f8-a663-11ec-aebe-f8e4e3a62943 comming from CESAR
Person 7307cf73-a663-11ec-b53b-f8e4e3a62943 comming from CESAR
Person 7306e362-a663-11ec-a363-f8e4e3a62943 comming from CESAR
Person 730843bd-a663-11ec-8458-f8e4e3a62943 comming from CESAR
Person 730757ea-a663-11ec-8c35-f8e4e3a62943 comming from CESAR
Person 73073114-a663-11ec-aeb6-f8e4e3a62943 comming from CESAR
Person 7306bc36-a663-11ec-af8b-f8e4e3a62943 comming from CESAR
Person 730709ef-a663-11ec-a887-f8e4e3a62943 comming from CESAR
Person 73077ec3-a663-11ec-ba1f-f8e4e3a62943 comming from CESAR
Person 73086abb-a663-11ec-a263-f8e4e3a62943 comming from CESAR
Person 7307f386-a663-11ec-ba8a-f8e4e3a62943 comming from CESAR
Person 7306e34d-a663-11ec-90d8-f8e4e3a62943 comming from CESAR
Person 730757ad-a663-11ec-9eab-f8e4e3a62943 co

Person 73135178-a663-11ec-875b-f8e4e3a62943 comming from CAQUETA
Person 73130416-a663-11ec-9fe6-f8e4e3a62943 comming from CAQUETA
Person 731351b8-a663-11ec-9d9c-f8e4e3a62943 comming from CAQUETA
Person 73130384-a663-11ec-9997-f8e4e3a62943 comming from CAQUETA
Person 73135222-a663-11ec-9f4f-f8e4e3a62943 comming from CAQUETA
Person 731351fa-a663-11ec-989e-f8e4e3a62943 comming from CAQUETA
Person 731303e9-a663-11ec-b871-f8e4e3a62943 comming from CAQUETA
Person 73132aa3-a663-11ec-b8f6-f8e4e3a62943 comming from CAQUETA
Person 731303ae-a663-11ec-a4a9-f8e4e3a62943 comming from CAQUETA
Person 73132aa9-a663-11ec-b557-f8e4e3a62943 comming from CAQUETA
Person 731303a2-a663-11ec-bdf2-f8e4e3a62943 comming from CAQUETA
Person 731351b1-a663-11ec-ba3e-f8e4e3a62943 comming from CAQUETA
Person 7312dcb8-a663-11ec-a37b-f8e4e3a62943 comming from CAQUETA
Person 731303ab-a663-11ec-b19a-f8e4e3a62943 comming from CAQUETA
Person 73132b30-a663-11ec-bd95-f8e4e3a62943 comming from CAQUETA
Person 731351f7-a663-11ec

Person 731378bb-a663-11ec-aab3-f8e4e3a62943 comming from PUTUMAYO
Person 7313a00b-a663-11ec-ad55-f8e4e3a62943 comming from PUTUMAYO
Person 7313c650-a663-11ec-ab82-f8e4e3a62943 comming from PUTUMAYO
Person 7313c696-a663-11ec-86a3-f8e4e3a62943 comming from PUTUMAYO
Person 7313a022-a663-11ec-8e27-f8e4e3a62943 comming from PUTUMAYO
Person 7313a050-a663-11ec-a07d-f8e4e3a62943 comming from PUTUMAYO
Person 7313a076-a663-11ec-9f97-f8e4e3a62943 comming from PUTUMAYO
Person 7313c678-a663-11ec-a2f4-f8e4e3a62943 comming from PUTUMAYO
Person 7313c687-a663-11ec-a694-f8e4e3a62943 comming from PUTUMAYO
Person 73139c26-a663-11ec-97ad-f8e4e3a62943 comming from PUTUMAYO
Person 73139fe5-a663-11ec-b6ea-f8e4e3a62943 comming from PUTUMAYO
Person 73137899-a663-11ec-9fd7-f8e4e3a62943 comming from PUTUMAYO
Person 73137874-a663-11ec-9e30-f8e4e3a62943 comming from PUTUMAYO
Person 7313c679-a663-11ec-9282-f8e4e3a62943 comming from PUTUMAYO
Person 73137856-a663-11ec-9e61-f8e4e3a62943 comming from PUTUMAYO
Person 731

Person 73141aec-a663-11ec-bcc9-f8e4e3a62943 comming from ARAUCA
Person 73141a88-a663-11ec-8f62-f8e4e3a62943 comming from ARAUCA
Person 7313ef99-a663-11ec-89a1-f8e4e3a62943 comming from ARAUCA
Person 73141ab4-a663-11ec-bc34-f8e4e3a62943 comming from ARAUCA
Person 73141aac-a663-11ec-8a84-f8e4e3a62943 comming from ARAUCA
Person 73141aea-a663-11ec-b9de-f8e4e3a62943 comming from ARAUCA
Person 7313ef30-a663-11ec-8a38-f8e4e3a62943 comming from ARAUCA
Person 73143e87-a663-11ec-bf49-f8e4e3a62943 comming from ARAUCA
Person 73141a45-a663-11ec-be2d-f8e4e3a62943 comming from ARAUCA
Person 73141ad7-a663-11ec-818c-f8e4e3a62943 comming from ARAUCA
Person 7313ef17-a663-11ec-8c9c-f8e4e3a62943 comming from ARAUCA
1400
estoy en el paso 55
1400
estoy en el paso 56
1400
estoy en el paso 57
1400
estoy en el paso 58
1400
estoy en el paso 59
1400
estoy en el paso 60
1400
estoy en el paso 61
1400
estoy en el paso 62
1400
estoy en el paso 63
1400
estoy en el paso 64
1400
estoy en el paso 65
1400
estoy en el paso

In [82]:
len(personas_def)

49842

In [9]:
#Subsample de personas_def para hacer pruebas 
tam=20
grupito=random.sample(personas_def, tam)
#for persona in grupito:
#    print(persona.origin.name)
grupito[1].tripRemaining

12.80666667

In [10]:
#for city in cities:
#   print(city.name)
#   print(city.machine)
#    print(city.population)
#    print('-----')

n_pasos = 5
delta = 0.5 #tiempo de cada paso en horas
#ciudad de origen de las personas 
for persona in grupito:
    print(persona.origin.name)

treated_people=[] 
#ejecución de la simulación 
print('resultado de la simulación')
for i in range(n_pasos):
    mundo=World(delta*(i+1),grupito)
    bla=mundo.step()

    
print(len(treated_people))

print('*********')



#print('las personas visitaron las siguientes ciudades')
#ciudades visitadas por cada persona 
#for persona in grupito:
#    a=persona.visited
#    print('----------')
#    print(persona)
#    for i in a:
#        print(i.name)
        



NARIÑO
NORTE DE SANTANDER
SANTANDER
NORTE DE SANTANDER
NORTE DE SANTANDER
HUILA
BOYACA
MAGDALENA
BOYACA
CORDOBA
CHOCO
TOLIMA
CUNDINAMARCA
SANTANDER
PUTUMAYO
MAGDALENA
SANTANDER
TOLIMA
CAQUETA
BOLIVAR
resultado de la simulación
5
*********


In [150]:
#step original
def step(self):
        for person in self.people:
            if person.tripRemaining > 0: 
                person.status = SEEKING_MACHINE
                person.tripRemaining = - self.delta
            else:
                cityD = person.destination
                person.visited.append(cityD)

                if cityD.machine is not None: #si llega al destino se aplica este condicional
                    if len(cityD.machine.treated) < LIMIT:
                        person.status = TREATED #cambia el estado si hay cupos en la máquina
                        cityD.machine.treated.append(person)
                        treated.append(person)#revisar para qué sirve 
                        if len(cityD.machine.treated) == LIMIT: #condicional para decidir qué pasa cuando se llena la máquina
                            cityD.machine.status = OCCUPIED
                            new_ori=cityD.name
                            person.origin = new_ori
                            tablaf_newD = tablasCorr(new_ori)
                            time_d=min(tablaf_newD)
                            n_ciudad_destino=tablaf_newD[tablaf_newD==time_d].index.values[0].split('s ')[1]    
                            
                            
                else:
                    #ir para otro lado si la ciudad no tiene máquinas 
                    #cityD.machine.status = OCCUPIED
                    new_ori=cityD.name
                    person.origin = new_ori
                    tablaf_newD = tablasCorr(new_ori)
                    time_d=min(tablaf_newD)
                    n_ciudad_destino=tablaf_newD[tablaf_newD==time_d].index.values[0].split('s ')[1]
        return treated_people